# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

# Store Part I results into DataFrame
- Load the csv exported in Part I to a DataFrame

In [2]:
# Import cities csv file
path = "../output_data/cities.csv"
cities = pd.read_csv(path)
cities.head()

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
0,Mahébourg,MU,-20.4081,57.7000,82.40,74,75,11.50,1612112744
1,Albany,US,42.6001,-73.9662,16.00,52,99,2.44,1612112676
2,Castro,BR,-24.7911,-50.0119,72.27,80,99,8.46,1612112745
3,Paamiut,GL,61.9940,-49.6678,24.71,92,0,14.34,1612112745
4,Sørland,NO,67.6670,12.6934,32.00,51,20,9.22,1612112188


# Humidity Heatmap

- Configure gmaps.
- Use the Lat and Lng as locations and Humidity as the weight
- Add Heatmap layer to map.

In [3]:
# Config gmaps
gmaps.configure(api_key=g_key)

# Based on locations and humidity
locations = cities[['Latitude', 'Longitude']]
humidity = cities["Humidity (%)"]

# Figure gmaps
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer with gmaps using locations and humidity
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(cities["Humidity (%)"]),
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

- Narrow down the cities to fit weather conditions.
- Drop any rows will null values.

In [4]:
# Create vacation dataframe.  My dream location.  Little clouds, barely any wind.  Temperature between 85-70
vacation_df= cities.loc[(cities['Max Temperature (F)']<=85) & (cities['Max Temperature (F)']>=70) 
                        & (cities['Wind Speed (mph)']<=5) 
                        & (cities['Cloudiness (%)']<=10)]

# Drop null values
vacation_df = vacation_df.dropna()

# Print df
vacation_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
96,Port Alfred,ZA,-33.5906,26.8910,73.00,78,3,1.99,1612112758
164,Wattegama,LK,6.7989,81.4808,73.24,90,0,4.81,1612112192
243,Makkah al Mukarramah,SA,21.4267,39.8261,76.06,44,0,4.27,1612112780
339,Banyo,CM,6.7500,11.8167,81.68,21,0,1.52,1612112793
355,Kruisfontein,ZA,-34.0033,24.7314,73.99,70,0,4.00,1612112795
357,Coahuayana Viejo,MX,18.7333,-103.6833,80.60,54,1,2.66,1612112796
440,Kumbo,CM,6.2000,10.6667,73.72,24,0,1.25,1612112808


# Hotel Map

- Store into variable named hotel_df.
- Add a "Hotel Name" column to the DataFrame.
- Set parameters to search for hotels with 5000 meters.
- Hit the Google Places API for each city's coordinates.
- Store the first Hotel result into the DataFrame.
- Plot markers on top of the heatmap.

In [6]:
# Rename vacation df to hotel_df.  Add hotel name column
hotel_df = vacation_df
hotel_df["Hotel Name"] = ""

# Create params
target_radius = 5000
target_type = "lodging"
gkey = g_key
keyword = "hotel"

# Set params
params = {"radius": target_radius,
    "type": target_type,
    "key": gkey,
      "keyword" : keyword}

# Generate hotels and store in hotel_df
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.


In [7]:
# Visualize to confirm hotel name appears
hotel_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date,Hotel Name
96,Port Alfred,ZA,-33.5906,26.8910,73.00,78,3,1.99,1612112758,MyPond Hotel
164,Wattegama,LK,6.7989,81.4808,73.24,90,0,4.81,1612112192,
243,Makkah al Mukarramah,SA,21.4267,39.8261,76.06,44,0,4.27,1612112780,Tera Sedky Hotel
339,Banyo,CM,6.7500,11.8167,81.68,21,0,1.52,1612112793,Hôtel Municipal de Banyo
355,Kruisfontein,ZA,-34.0033,24.7314,73.99,70,0,4.00,1612112795,
357,Coahuayana Viejo,MX,18.7333,-103.6833,80.60,54,1,2.66,1612112796,Hotel Aeropuerto
440,Kumbo,CM,6.2000,10.6667,73.72,24,0,1.25,1612112808,Motel La Residence


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)

fig = gmaps.figure(map_type="HYBRID")


markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add layer
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))